In [1]:
import db_conn
import pandas as pd
import numpy as np
import collections
from IPython.display import display

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)

In [2]:
db_conn.select_query('show tables')

[{'Tables_in_football_worldcup': 'player'},
 {'Tables_in_football_worldcup': 'player_stat'},
 {'Tables_in_football_worldcup': 'stat_code'},
 {'Tables_in_football_worldcup': 'team'},
 {'Tables_in_football_worldcup': 'team_play_stat'},
 {'Tables_in_football_worldcup': 'wc_match'}]

In [3]:
wc_match = pd.DataFrame(db_conn.select_query('select * from wc_match'))

In [4]:
wc_match.head()

,away_team_id,away_team_score,home_team_id,home_team_score,id,match_date,match_location
0,2,0,1,5,1,14 JUN 2018 18:00,Moscow / Luzhniki Stadium / RUS
1,4,1,3,0,2,15 JUN 2018 17:00,Ekaterinburg / Ekaterinburg Arena / RUS
2,6,3,5,3,3,15 JUN 2018 21:00,Sochi / Fisht Olympic Stadium / RUS
3,8,1,7,0,4,15 JUN 2018 18:00,Saint Petersburg /
4,10,1,9,2,5,16 JUN 2018 13:00,Kazan / Kazan Arena / RUS


In [5]:
team = pd.DataFrame(db_conn.select_query('select * from team'))

In [6]:
team.head()

,country,id
0,Russia,1
1,Saudi Arabia,2
2,Egypt,3
3,Uruguay,4
4,Portugal,5


In [35]:
players = pd.DataFrame(db_conn.select_query('select p.team_id, p.num, p.name, p.club, p.position, ps.* from player_stat ps left join player p on ps.player_id = p.id'))

In [36]:
players.head()

,club,half,id,match_id,name,num,player_id,position,stat_type,stat_value,team_id
0,Al Hilal (KSA),total,1,1,ABDULLAH ALMUAIOUF,1,1,GK,POS,Goalkeeper,2
1,Al Hilal (KSA),2,2,1,ABDULLAH ALMUAIOUF,1,1,GK,SAVE,2,2
2,Al Hilal (KSA),1,3,1,ABDULLAH ALMUAIOUF,1,1,GK,DAT,1,2
3,Al Hilal (KSA),1,4,1,ABDULLAH ALMUAIOUF,1,1,GK,CC,2,2
4,Al Hilal (KSA),1,5,1,ABDULLAH ALMUAIOUF,1,1,GK,CA,2,2


In [2]:
import mongo_conn
from pymongo import MongoClient

In [110]:
documents = []
for match in wc_match.iterrows():
    document = {}
    home_players = list((v.dropna().to_dict()) for k, v in players[(players.match_id == match[1]['id']) & (players.team_id == match[1]['home_team_id']) & (players.half == 'total') & \
        (~players.stat_type.isin(['POS', 'DZONE1', 'DZONE2', 'DZONE3', 'DZONE4', 'DZONE5', 'TZONE1', 'TZONE2', 'TZONE3', 'TZONE4', 'TZONE5']))]\
.pivot_table(values='stat_value', index=['num', 'name', 'club', 'position'], columns='stat_type', aggfunc='first')\
.reset_index().iterrows())
    away_players = list((v.dropna().to_dict()) for k, v in players[(players.match_id == match[1]['id']) & (players.team_id == match[1]['away_team_id']) & (players.half == 'total') & \
        (~players.stat_type.isin(['POS', 'DZONE1', 'DZONE2', 'DZONE3', 'DZONE4', 'DZONE5', 'TZONE1', 'TZONE2', 'TZONE3', 'TZONE4', 'TZONE5']))]\
.pivot_table(values='stat_value', index=['num', 'name', 'club', 'position'], columns='stat_type', aggfunc='first')\
.reset_index().iterrows())
    
    document['date'] = match[1]['match_date']    
    document['home_team'] = {
        'country': team[team.id == match[1]['home_team_id']].country.values[0], 
        'score': match[1]['home_team_score'],
        'players': home_players
    }
    document['away_team'] = {
        'country': team[team.id == match[1]['away_team_id']].country.values[0], 
        'score': match[1]['away_team_score'],
        'players': away_players
    }
    documents.append(document)

In [111]:
len(documents)

64

In [3]:
client = mongo_conn.get_client()

In [104]:
football_database = client.Football

In [105]:
match_collection = football_database.wc_match

In [113]:
for d in documents:
    match_collection.insert(d)

/Users/gracelee/.pyenv/versions/for_jupyter/lib/python3.5/site-packages/ipykernel_launcher.py:2: DeprecationWarning: insert is deprecated. Use insert_one or insert_many instead.
  
